In [1]:
%%writefile mapper-tf.py
from __future__ import print_function
import sys
import re


reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

path = 'stop_words_en.txt'

stopWords = set()

with open(path) as stopWordsFile:
    for line in stopWordsFile:
        try:         
            stopWords.add(unicode(line.strip()))
        except ValueError as e:
            continue

import string

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue

    words = re.split("\W*\s+\W*", text.lower(), flags=re.UNICODE)
    
    wordsInArticle = 0
    
    for word in words:
        word = word.strip(string.punctuation)
        if word in stopWords:
            print("reporter:counter:Wiki stats,Stop words found,%d" % 1, file=sys.stderr)
            continue
        if word == "":
            print("reporter:counter:Wiki stats,Null words found,%d" % 1, file=sys.stderr)
            continue
        wordsInArticle += 1
        print("reporter:counter:Wiki stats,Total words found,%d" % 1, file=sys.stderr)
        print(article_id, word.strip(string.punctuation), 1, sep='\t')
    print(article_id, "!wordsInArticle", wordsInArticle, sep='\t')



Writing mapper-tf.py


In [2]:
%%writefile reducer-tf.py
import sys


reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

current_article_id = None
current_word = None
word_sum, wordsInArticle = 0, 0

for line in sys.stdin:
    try:
        article_id, word, count = unicode(line.strip()).split('\t')
        #print "after parse:", article_id, word, count
        if article_id == "" or word == "" or count == "":   # sanity check
            continue
        count = int(count)
    except ValueError as e:
        #print "Parse error:", e
        continue

    if current_article_id != article_id:
        if current_word != None:
            print "%s\t%s\t%f" % (current_article_id, current_word, 1.0*word_sum/wordsInArticle)
            word_sum = 0
            current_word = word
        wordsInArticle = count  # first article string is always words in article count
        current_article_id = article_id
        #print "wordsInArticle:", word, wordsInArticle
        continue
        
    if current_word != word :
        if current_word != None and current_word != '!wordsInArticle': 
            print "%s\t%s\t%f" % (current_article_id, current_word, 1.0*word_sum/wordsInArticle)
            #print "%s\t%s\t%d\t%d" % (current_article_id, current_word, word_sum, wordsInArticle)
        word_sum = 0
        current_word = word
    word_sum += count

if current_article_id:
    print "%s\t%s\t%f" % (current_article_id, word, 1.0*word_sum/wordsInArticle)


Writing reducer-tf.py


In [3]:
!hdfs dfs -mkdir /data/wiki/test

In [4]:
!hdfs dfs -mkdir /data/wiki/test/data

In [5]:
!hdfs dfs -put wiki-test.txt /data/wiki/test/data

In [6]:
!hdfs dfs -ls /data/wiki/test

Found 1 items
drwxr-xr-x   - jovyan supergroup          0 2019-02-06 08:57 /data/wiki/test/data


In [7]:
!hdfs dfs -ls /data/wiki/test/data

Found 1 items
-rw-r--r--   1 jovyan supergroup        479 2019-02-06 08:57 /data/wiki/test/data/wiki-test.txt


In [8]:
%%bash

#OUT_DIR="wordcount_result_"$(date +"%s%6N")
OUT_DIR="/data/wiki/test/tf-out"
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.job.output.key.partitioner.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedPartitioner \
    -D mapreduce.job.name="Tf-Idf tf task" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper-tf.py,reducer-tf.py,/datasets/stop_words_en.txt  \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.partition.keypartitioner.options="-k1,1" \
    -D mapreduce.partition.keycomparator.options="-k1,2" \
    -mapper "python mapper-tf.py" \
    -reducer "python reducer-tf.py" \
    -input /data/wiki/test/data \
    -output ${OUT_DIR} > /dev/null


hdfs dfs -cat ${OUT_DIR}/* | head -100

10	3	0.153846
10	behavior	0.076923
10	congrayou	0.076923
10	emulate	0.076923
10	hadoop	0.153846
10	hadoop's	0.076923
10	job	0.153846
10	know	0.076923
10	stages	0.153846
17	0	0.062500
17	article_ids	0.062500
17	bash	0.062500
17	behavior	0.062500
17	code	0.062500
17	emulate	0.062500
17	greater	0.062500
17	hadoop's	0.062500
17	hint	0.125000
17	mapper's	0.062500
17	reducer's	0.062500
17	simple	0.062500
17	using	0.062500
17	wikipedia	0.125000
23	applications	0.125000
23	bash	0.062500
23	congrayou	0.125000
23	debugging	0.125000
23	easily	0.062500
23	emulating	0.062500
23	hadoop’s	0.062500
23	locally	0.062500
23	lot	0.062500
23	safe	0.062500
23	time	0.062500
23	using	0.062500
23	workflow	0.062500


rm: `/data/wiki/test/tf-out': No such file or directory
19/02/06 08:58:45 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/06 08:58:45 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/06 08:58:46 INFO mapred.FileInputFormat: Total input files to process : 1
19/02/06 08:58:47 INFO mapreduce.JobSubmitter: number of splits:2
19/02/06 08:58:47 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1549443035368_0001
19/02/06 08:58:48 INFO impl.YarnClientImpl: Submitted application application_1549443035368_0001
19/02/06 08:58:48 INFO mapreduce.Job: The url to track the job: http://427e53c98640:8088/proxy/application_1549443035368_0001/
19/02/06 08:58:48 INFO mapreduce.Job: Running job: job_1549443035368_0001
19/02/06 08:58:55 INFO mapreduce.Job: Job job_1549443035368_0001 running in uber mode : false
19/02/06 08:58:55 INFO mapreduce.Job:  map 0% reduce 0%
19/02/06 08:59:01 INFO mapreduce.Job:  map 100% reduce 0%
19/02/06 08:59:07 INFO 

In [9]:
! hdfs dfs -ls /data/wiki/test/tf-out

Found 2 items
-rw-r--r--   1 jovyan supergroup          0 2019-02-06 08:59 /data/wiki/test/tf-out/_SUCCESS
-rw-r--r--   1 jovyan supergroup        701 2019-02-06 08:59 /data/wiki/test/tf-out/part-00000


In [10]:
%%writefile mapper-idf.py
from __future__ import print_function
import sys

reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

for line in sys.stdin:
    _, word, _ = unicode(line.strip()).split('\t')
    print("reporter:counter:Idf stats,Total words found,%d" % 1, file=sys.stderr)
    print(word, 1, sep='\t')


Writing mapper-idf.py


In [11]:
%%writefile reducer-idf.py
import sys
from math import log

reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

current_key = None
word_sum = 0
for line in sys.stdin:
    try:
        key, count = unicode(line.strip()).split('\t', 1)
        if key == "" or count == "":    # sanity check
            continue
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%f" % (current_key, (1/log(1 + word_sum)))
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%f" % (current_key, (1/log(1 + word_sum)))

Writing reducer-idf.py


In [12]:
%%bash

#OUT_DIR="wordcount_result_"$(date +"%s%6N")
OUT_DIR="/data/wiki/test/idf-out"
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Tf-Idf idf task" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper-idf.py,reducer-idf.py \
    -mapper "python mapper-idf.py" \
    -reducer "python reducer-idf.py" \
    -input /data/wiki/test/tf-out \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/* 

0	1.442695
3	1.442695
applications	1.442695
article_ids	1.442695
bash	0.910239
behavior	0.910239
code	1.442695
congrayou	0.910239
debugging	1.442695
easily	1.442695
emulate	0.910239
emulating	1.442695
greater	1.442695
hadoop	1.442695
hadoop's	0.910239
hadoop’s	1.442695
hint	1.442695
job	1.442695
know	1.442695
locally	1.442695
lot	1.442695
mapper's	1.442695
reducer's	1.442695
safe	1.442695
simple	1.442695
stages	1.442695
time	1.442695
using	0.910239
wikipedia	1.442695
workflow	1.442695


rm: `/data/wiki/test/idf-out': No such file or directory
19/02/06 09:01:07 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/06 09:01:07 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/06 09:01:09 INFO mapred.FileInputFormat: Total input files to process : 1
19/02/06 09:01:09 INFO mapreduce.JobSubmitter: number of splits:2
19/02/06 09:01:09 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1549443035368_0002
19/02/06 09:01:09 INFO impl.YarnClientImpl: Submitted application application_1549443035368_0002
19/02/06 09:01:09 INFO mapreduce.Job: The url to track the job: http://427e53c98640:8088/proxy/application_1549443035368_0002/
19/02/06 09:01:09 INFO mapreduce.Job: Running job: job_1549443035368_0002
19/02/06 09:01:14 INFO mapreduce.Job: Job job_1549443035368_0002 running in uber mode : false
19/02/06 09:01:14 INFO mapreduce.Job:  map 0% reduce 0%
19/02/06 09:01:20 INFO mapreduce.Job:  map 100% reduce 0%
19/02/06 09:01:26 INFO

In [13]:
!hdfs dfs -ls /data/wiki/test/idf-out

Found 2 items
-rw-r--r--   1 jovyan supergroup          0 2019-02-06 09:01 /data/wiki/test/idf-out/_SUCCESS
-rw-r--r--   1 jovyan supergroup        492 2019-02-06 09:01 /data/wiki/test/idf-out/part-00000


In [14]:
%%writefile mapper-tf-idf.py
from __future__ import print_function
import sys


reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode


for line in sys.stdin:
    inVal = unicode(line.strip()).split('\t')
    inValLen = len(inVal)
    #print(inValLen, inVal)
    try:
        if inValLen == 2: # idf data
            print("reporter:counter:Wiki stats,idf words,%d" % 1, file=sys.stderr)
            print(inVal[0], "idf", float(inVal[1]), sep='\t')
        elif inValLen == 3: # tf data
            print("reporter:counter:Wiki stats,tf words,%d" % 1, file=sys.stderr)
            print(inVal[1], "tf", inVal[0], float(inVal[2]), sep='\t')
        else:
            print("reporter:counter:Wiki stats,tf idf parsing errors,%d" % 1, file=sys.stderr)
    except ValueError as e:
        print("reporter:counter:Wiki stats,tf idf to float errors,%d" % 1, file=sys.stderr)


Writing mapper-tf-idf.py


In [15]:
%%writefile reducer-tf-idf.py
import sys
from math import log


reload(sys)
sys.setdefaultencoding('utf-8')  # required to convert to unicode

for line in sys.stdin:
    inVal = unicode(line.strip()).split('\t')
    if len(inVal) < 3 or inVal[0] == "" or inVal[1] == "" or inVal[2] == "":      # sanity check
        continue
    try:
        if inVal[1] == 'idf':
            idfWord, _, idfDt = inVal
            idfWord = idfWord
            idfDt = float(idfDt)
            continue
        elif inVal[1] == 'tf':
            tfWord, _, article_id, tfNt = inVal
            tfWord = tfWord
            tfNt = float(tfNt)
        else:
            continue
    except ValueError as e:
        continue
    if idfWord == tfWord:
        print "%s\t%s\t%f" % (idfWord, article_id, tfNt*idfDt)


Writing reducer-tf-idf.py


In [16]:
%%bash

#OUT_DIR="wordcount_result_"$(date +"%s%6N")
OUT_DIR="/data/wiki/test/tf-idf-out"
NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.job.output.key.partitioner.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedPartitioner \
    -D mapreduce.job.name="Tf-Idf tf*idf task" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper-tf-idf.py,reducer-tf-idf.py \
    -D mapreduce.partition.keypartitioner.options="-k1,1" \
    -D stream.num.map.output.key.fields=2 \
    -mapper "python mapper-tf-idf.py" \
    -reducer "python reducer-tf-idf.py" \
    -input /data/wiki/test/tf-out,/data/wiki/test/idf-out \
    -output ${OUT_DIR} > /dev/null

hdfs dfs -cat ${OUT_DIR}/* 

0	17	0.090168
3	10	0.221953
applications	23	0.180337
article_ids	17	0.090168
bash	17	0.056890
bash	23	0.056890
behavior	17	0.056890
behavior	10	0.070018
code	17	0.090168
congrayou	10	0.070018
congrayou	23	0.113780
debugging	23	0.180337
easily	23	0.090168
emulate	10	0.070018
emulate	17	0.056890
emulating	23	0.090168
greater	17	0.090168
hadoop	10	0.221953
hadoop's	17	0.056890
hadoop's	10	0.070018
hadoop’s	23	0.090168
hint	17	0.180337
job	10	0.221953
know	10	0.110976
locally	23	0.090168
lot	23	0.090168
mapper's	17	0.090168
reducer's	17	0.090168
safe	23	0.090168
simple	17	0.090168
stages	10	0.221953
time	23	0.090168
using	23	0.056890
using	17	0.056890
wikipedia	17	0.180337
workflow	23	0.090168


rm: `/data/wiki/test/tf-idf-out': No such file or directory
19/02/06 09:03:36 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/06 09:03:36 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/02/06 09:03:38 INFO mapred.FileInputFormat: Total input files to process : 2
19/02/06 09:03:38 INFO mapreduce.JobSubmitter: number of splits:3
19/02/06 09:03:38 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1549443035368_0003
19/02/06 09:03:39 INFO impl.YarnClientImpl: Submitted application application_1549443035368_0003
19/02/06 09:03:39 INFO mapreduce.Job: The url to track the job: http://427e53c98640:8088/proxy/application_1549443035368_0003/
19/02/06 09:03:39 INFO mapreduce.Job: Running job: job_1549443035368_0003
19/02/06 09:03:45 INFO mapreduce.Job: Job job_1549443035368_0003 running in uber mode : false
19/02/06 09:03:45 INFO mapreduce.Job:  map 0% reduce 0%
19/02/06 09:03:51 INFO mapreduce.Job:  map 100% reduce 0%
19/02/06 09:03:58 I

In [19]:
!hdfs dfs -ls /data/wiki/test/tf-idf-out

Found 4 items
-rw-r--r--   1 jovyan supergroup          0 2019-02-05 07:24 /data/wiki/test/tf-idf-out/_SUCCESS
-rw-r--r--   1 jovyan supergroup       1086 2019-02-05 07:24 /data/wiki/test/tf-idf-out/part-00000
-rw-r--r--   1 jovyan supergroup        861 2019-02-05 07:24 /data/wiki/test/tf-idf-out/part-00001
-rw-r--r--   1 jovyan supergroup        164 2019-02-05 07:24 /data/wiki/test/tf-idf-out/part-00002


In [17]:
!hdfs dfs -cat /data/wiki/test/tf-idf-out/part-00000

0	17	0.090168
3	10	0.221953
applications	23	0.180337
article_ids	17	0.090168
bash	17	0.056890
bash	23	0.056890
behavior	17	0.056890
behavior	10	0.070018
code	17	0.090168
congrayou	10	0.070018
congrayou	23	0.113780
debugging	23	0.180337
easily	23	0.090168
emulate	10	0.070018
emulate	17	0.056890
emulating	23	0.090168
greater	17	0.090168
hadoop	10	0.221953
hadoop's	17	0.056890
hadoop's	10	0.070018
hadoop’s	23	0.090168
hint	17	0.180337
job	10	0.221953
know	10	0.110976
locally	23	0.090168
lot	23	0.090168
mapper's	17	0.090168
reducer's	17	0.090168
safe	23	0.090168
simple	17	0.090168
stages	10	0.221953
time	23	0.090168
using	23	0.056890
using	17	0.056890
wikipedia	17	0.180337
workflow	23	0.090168
